In [64]:
table_product = """CREATE TABLE Product (
                      part TEXT PRIMARY KEY, 
                      part_name TEXT NOT NULL,
                      weight REAL
                      )"""

table_cost = """CREATE TABLE Cost (
                  part TEXT,
                  cost_date DATETIME NOT NULL,
                  cost REAl NOT NULL,
                  FOREIGN KEY (part) REFERENCES Product (part)
                  )"""

table_stock = """CREATE TABLE Stock (
                  part TEXT,
                  operation_date DATETIME NOT NULL,
                  qty INTEGER NOT NULL,
                  cost REAL  NOT NULL,
                  operation_type TEXT NOT NULL,
                  FOREIGN KEY (part) REFERENCES Product (part)
                  )"""

In [65]:
# створюємо таблиці в БД
import sqlite3
conn = sqlite3.connect('store.db')

curs = conn.cursor()

curs.execute(table_product)
curs.execute(table_cost)
curs.execute(table_stock)

conn.close()

In [66]:
# вносимо інформацію в таблицю Prodcut

conn = sqlite3.connect('store.db')
curs = conn.cursor()
sql_text = """INSERT INTO Product VALUES(?, ?, ?)"""

part = input('Введіть артикул товару')
part_name = input('Введіть назву товару')
weight = float(input('Введіть вагу товару'))

curs.execute(sql_text, (part, part_name, weight))
conn.commit()

conn.close()

Введіть артикул товаруre509672
Введіть назву товаруFilter
Введіть вагу товару300


In [67]:
conn = sqlite3.connect('store.db')
curs = conn.cursor()

sql_text = "SELECT * FROM Product"

curs.execute(sql_text)

for el in curs.fetchall():
    print(el)


conn.close()

('re509672', 'Filter', 300.0)


In [69]:
#записуємо ціну
import datetime

conn = sqlite3.connect('store.db')
curs = conn.cursor()
sql_text = """INSERT INTO Cost VALUES(?, ?, ?)"""

part = input('Введіть артикул товару')
date = datetime.datetime.now()
cost = float(input('Введіть ціну товару'))

curs.execute(sql_text, (part, date, cost))
conn.commit()

conn.close()

Введіть артикул товаруre509672
Введіть ціну товару400


In [72]:
#записуємо прихід\розхід

import datetime

conn = sqlite3.connect('store.db')
curs = conn.cursor()


part = input('Введіть артикул товару')
date = datetime.datetime.now()
qty = int(input('Введіть кількість товару'))
operation_type = input('Введіть тип операції: 1-прихід || 2-розхід')

curs.execute("SELECT cost, cost_date FROM Cost WHERE part = (?) ORDER BY cost_date ", (part,) )

cost = curs.fetchall()[-1][0]


sql_text = """INSERT INTO Stock VALUES(?, ?, ?, ?, ?)"""
curs.execute(sql_text, (part, date, qty, cost, operation_type))
conn.commit()

conn.close()

Введіть артикул товаруre509672
Введіть кількість товару1
Введіть тип операції: 1-прихід || 2-розхід2


In [18]:
import requests 
from bs4 import BeautifulSoup
r = requests.get('https://smallpacking.agrosem.ua/shop/')
soup = BeautifulSoup(r.text, 'html.parser')

all_products = {}

def get_soup(href):
    r = requests.get(href)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

def product_informations(base, href):
    p = get_html_text(href)
    sku = p.find('span', class_='sku')
    print(sku)
    
    
def all_products_page(page_text):
    l = []
    for el in soup.find_all('div', class_="product-from-category-container"):
        l.append(el.find('a').get('href'))
    return l
        
def all_page(href):
    p = get_html_text(href)
    list_with_product = all_products_page(p)
    print(list_with_product)
    
all_page('https://smallpacking.agrosem.ua/shop/')
    

['https://smallpacking.agrosem.ua/product/dlya-vinogradu/', 'https://smallpacking.agrosem.ua/product/dlya-vichnozelenih-derev-ta-kushhiv/', 'https://smallpacking.agrosem.ua/product/dlya-vichnozelenih-derev-ta-kushhiv-3/', 'https://smallpacking.agrosem.ua/product/dlya-vichnozelenih-derev-ta-kushhiv-4/', 'https://smallpacking.agrosem.ua/product/dlya-gazonu-5/', 'https://smallpacking.agrosem.ua/product/dlya-gazonu-6/', 'https://smallpacking.agrosem.ua/product/dlya-gazonu/', 'https://smallpacking.agrosem.ua/product/dlya-dekorativnih-listovih-3/', 'https://smallpacking.agrosem.ua/product/dlya-poluniczi/', 'https://smallpacking.agrosem.ua/product/dlya-dekorativnih-listovih/', 'https://smallpacking.agrosem.ua/product/dlya-ovochiv-na-vidkritomu-grunti/', 'https://smallpacking.agrosem.ua/product/dlya-ovochiv-na-vidkritomu-grunti-2/', 'https://smallpacking.agrosem.ua/product/dlya-rozsadi/', 'https://smallpacking.agrosem.ua/product/dlya-teplichnih-ovochiv/', 'https://smallpacking.agrosem.ua/produ

In [35]:
import requests 
from bs4 import BeautifulSoup
r = requests.get('https://smallpacking.agrosem.ua/shop/')
soup = BeautifulSoup(r.text, 'html.parser')

all_products = {}

# def get_soup(href):
#     r = requests.get(href)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     return soup
for i in range(1, 4):
    r = requests.get(f'https://smallpacking.agrosem.ua/shop/page/{i}/')
    soup = BeautifulSoup(r.text, 'html.parser')
    for el in soup.find_all('div', class_="product-from-category-container"):
        href = el.find('a').get('href')
        r = requests.get(href)
        soup = BeautifulSoup(r.text, 'html.parser')
        sku = soup.find('span', class_='sku').text.strip('SKU: ')
        name = soup.find('div', class_="atributes-description").find('h2').text
        price = float(soup.find('span', class_='regular-price').text.rstrip(' грн'))
        p = soup.find('span', class_="type-packaging-name").text
        if p == 'Коробка':
            weight = 1
        elif p == 'Мішок':
            weight = 3
        elif p == 'Відро':
            weight = 10
        else:
            weight = None

        all_products[sku] = {'name':name, 'price':price, 'weight':weight}
        
        
all_products

{'797756': {'name': 'Добриво для винограду', 'price': 150.0, 'weight': 1},
 '8362626': {'name': 'Добриво для вічнозелених дерев та кущів',
  'price': 150.0,
  'weight': 1},
 '4738184': {'name': 'Добриво для вічнозелених дерев та кущів',
  'price': 360.0,
  'weight': 3},
 '5218': {'name': 'Добриво для вічнозелених дерев та кущів',
  'price': 995.0,
  'weight': 10},
 '7143174': {'name': 'Добриво для газону', 'price': 360.0, 'weight': 3},
 '375738': {'name': 'Добриво для газону', 'price': 995.0, 'weight': 10},
 '789868': {'name': 'Добриво для газону', 'price': 150.0, 'weight': 1},
 '78747': {'name': 'Добриво для декоративних листових',
  'price': 360.0,
  'weight': 3},
 '3593': {'name': 'Добриво для полуниці', 'price': 150.0, 'weight': 1},
 '33486': {'name': 'Добриво для декоративних листових',
  'price': 150.0,
  'weight': 1},
 '198742': {'name': 'Добриво для овочів', 'price': 360.0, 'weight': 3},
 '88255': {'name': 'Добриво для овочів', 'price': 150.0, 'weight': 1},
 '68183': {'name': '

In [36]:
import sqlite3 
conn = sqlite3.connect('store.db')
curs = conn.cursor()
curs.execute('DELETE FROM Product')

sql_text = """INSERT INTO Product VALUES(?, ?, ?)"""


for k,v in all_products.items():
    curs.execute(sql_text, (k, v['name'], v['weight']))
    conn.commit()

conn.close()
    


In [38]:
import sqlite3 
conn = sqlite3.connect('store.db')
curs = conn.cursor()
curs.execute('SELECT * FROM Product')

for el in curs.fetchall():
    print(el)

conn.close()

('797756', 'Добриво для винограду', 1.0)
('8362626', 'Добриво для вічнозелених дерев та кущів', 1.0)
('4738184', 'Добриво для вічнозелених дерев та кущів', 3.0)
('5218', 'Добриво для вічнозелених дерев та кущів', 10.0)
('7143174', 'Добриво для газону', 3.0)
('375738', 'Добриво для газону', 10.0)
('789868', 'Добриво для газону', 1.0)
('78747', 'Добриво для декоративних листових', 3.0)
('3593', 'Добриво для полуниці', 1.0)
('33486', 'Добриво для декоративних листових', 1.0)
('198742', 'Добриво для овочів', 3.0)
('88255', 'Добриво для овочів', 1.0)
('68183', 'Добриво для розсади', 1.0)
('51561', 'Добриво для тепличних овочів', 1.0)
('5656841', 'Добриво для тепличних овочів', 3.0)
('2217672', 'Добриво для троянд', 1.0)
('439226', 'Добриво для фруктових та горіхових дерев', 3.0)
('3177412', 'Добриво для фруктових та горіхових дерев', 1.0)
('1297', 'Добриво для ягідних чагарників', 3.0)
('91897', 'Добриво для ягідних чагарників', 1.0)
('61169', 'Добриво універсальне грунтове', 1.0)
('6651824

In [42]:
import sqlite3 
from datetime import datetime, timedelta
import random
conn = sqlite3.connect('store.db')
curs = conn.cursor()


sql_text = """INSERT INTO Cost VALUES(?, ?, ?)"""


for k,v in all_products.items():
    curs.execute(sql_text, (k, datetime.now() + timedelta(days=random.randrange(100)), v['price']))
    conn.commit()

conn.close()

In [ ]:
 """CREATE TABLE Stock (
                  part TEXT,
                  operation_date DATETIME NOT NULL,
                  qty INTEGER NOT NULL,
                  cost REAL  NOT NULL,
                  operation_type TEXT NOT NULL,
                  FOREIGN KEY (part) REFERENCES Product (part)
                  )"""

In [45]:
import sqlite3 
from datetime import datetime, timedelta
import random
conn = sqlite3.connect('store.db')
curs = conn.cursor()


sql_text = """INSERT INTO Stock VALUES(?, ?, ?, ?, ?)"""

def product_cost(part):
    curs.execute("SELECT cost, cost_date FROM Cost WHERE part = (?) ORDER BY cost_date ", (part,) )
    cost = curs.fetchall()[-1][0]
    return cost

for k in all_products:
    for i in range(random.randrange(10)):
        curs.execute(sql_text, (k 
                                ,datetime.now() + timedelta(days=random.randrange(50))
                               , random.randrange(10, 20)
                               , product_cost(k)
                               , 'Розхід'))
        conn.commit()

conn.close()